In [1]:
import pandas as pd
import numpy as np
import utils
import xgboost as xgb

/home/mike/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import importlib
utils = importlib.reload(utils)

In [ ]:
!ls

In [58]:
df_bm25_unigram_scores = utils.load_from_file("df_bm25_unigram_scores.pkl")
df_bm25_unigram_scores_smooth = utils.load_from_file("df_bm25_unigram_scores_smooth.pkl")
df_bm25_bigram_scores = utils.load_from_file("df_bm25_bigram_scores.pkl")
df_bm25_bigram_scores_smooth = utils.load_from_file("df_bm25_bigram_scores_smooth.pkl")
df_bm25_trigram_scores = utils.load_from_file("df_bm25_trigram_scores.pkl")
df_bm25_trigram_scores_smooth = utils.load_from_file("df_bm25_trigram_scores_smooth.pkl")
df_coverage_scores = utils.load_from_file("df_coverage_scores.pkl")

In [59]:
test_data = df_bm25_unigram_scores
for another_data in [ df_bm25_unigram_scores_smooth,
                      df_bm25_bigram_scores,
                      df_bm25_bigram_scores_smooth,
                      df_bm25_trigram_scores,
                      df_bm25_trigram_scores_smooth,
                      df_coverage_scores]:
    for column in another_data.columns:
        if column=='query_id' or column=='doc_id':
            continue
        test_data[column] = another_data[column]

In [60]:
df_train_bm25_unigram_scores = utils.load_from_file("df_train_bm25_unigram_scores.pkl")
df_train_bm25_unigram_scores_smooth = utils.load_from_file("df_train_bm25_unigram_scores_smooth.pkl")
df_train_bm25_bigram_scores = utils.load_from_file("df_train_bm25_bigram_scores.pkl")
df_train_bm25_bigram_scores_smooth = utils.load_from_file("df_train_bm25_bigram_scores_smooth.pkl")
df_train_bm25_trigram_scores = utils.load_from_file("df_train_bm25_trigram_scores.pkl")
df_train_bm25_trigram_scores_smooth = utils.load_from_file("df_train_bm25_trigram_scores_smooth.pkl")
df_train_coverage_scores = utils.load_from_file("df_train_coverage_scores.pkl")

In [69]:
train_data = df_train_bm25_unigram_scores
for another_data in [ df_train_bm25_unigram_scores_smooth,
                      df_train_bm25_bigram_scores,
                      df_train_bm25_bigram_scores_smooth,
                      df_train_bm25_trigram_scores,
                      df_train_bm25_trigram_scores_smooth,
                      df_train_coverage_scores]:
    for column in another_data.columns:
        if column=='query_id' or column=='doc_id':
            continue
        train_data[column] = another_data[column]

In [70]:
test_data.columns

Index(['query_id', 'doc_id', 'score_title_bm25', 'score_body_bm25',
       'score_title_bm25_smooth', 'score_body_bm25_smooth',
       'score_title_bm25_bigram', 'score_body_bm25_bigram',
       'score_title_bm25_bigram_smooth', 'score_body_bm25_bigram_smooth',
       'score_title_bm25_trigram', 'score_body_bm25_trigram',
       'score_title_bm25_trigram_smooth', 'score_body_bm25_trigram_smooth',
       'title_count', 'body_count', 'whole_count', 'title_ratio', 'body_ratio',
       'whole_ratio', 'target'],
      dtype='object')

In [71]:
queries = utils.load_queries('data/queries_tr.tsv')
test_queries = utils.load_test('data/test.tsv')
train_queries = utils.load_train('data/clicks.train.tsv')

In [72]:
train_data['target'] = 0
test_data['target'] = 0

In [73]:
clicked_pairs = set()
for query_id, docs, clicks in train_queries:
    clicks_set = set(clicks)
    for doc_id in docs:
        if doc_id in clicks:
            #train_data.target[(train_data.query_id==query_id) & (train_data.doc_id==doc_id)] = 1
            clicked_pairs.add((query_id, doc_id))

In [74]:
train_data.target = train_data.apply(lambda x: (x['query_id'], x['doc_id']) in clicked_pairs, axis=1)

In [75]:
feature_columns = ['score_title_bm25', 'score_body_bm25',
       'score_title_bm25_smooth', 'score_body_bm25_smooth',
       'score_title_bm25_bigram', 'score_body_bm25_bigram',
       'score_title_bm25_bigram_smooth', 'score_body_bm25_bigram_smooth',
       'score_title_bm25_trigram', 'score_body_bm25_trigram',
       'score_title_bm25_trigram_smooth', 'score_body_bm25_trigram_smooth',
       'title_count', 'body_count', 'whole_count', 'title_ratio', 'body_ratio',
       'whole_ratio']

In [103]:
feature_columns = [#'score_title_bm25', 'score_body_bm25',
       #'score_title_bm25_smooth', 'score_body_bm25_smooth',
       #'score_title_bm25_bigram', 'score_body_bm25_bigram',
       #'score_title_bm25_bigram_smooth', 'score_body_bm25_bigram_smooth',
       'score_title_bm25_trigram', 'score_body_bm25_trigram',
       #'score_title_bm25_trigram_smooth', 'score_body_bm25_trigram_smooth',
       'title_count', 'body_count', 'whole_count', 'title_ratio', 'body_ratio',
       'whole_ratio']

In [104]:
dtrain = xgb.DMatrix(train_data[feature_columns], label=train_data.target)
train_groups = [len(docs) for query_id,docs,clicks in train_queries]
dtrain.set_group(train_groups)

In [105]:
dtest = xgb.DMatrix(test_data[feature_columns], label=test_data.target)
test_groups = [len(docs) for query_id,docs in test_queries]
dtest.set_group(test_groups)

In [106]:
param = {'max_depth':5, 'eta':1, 'silent':1, 'objective':'rank:pairwise',
         'eval_metric': ['ndcg@5']}

In [107]:
watchlist  = [(dtest,'eval'), (dtrain,'train')]

In [108]:
num_round = 500

In [109]:
model = xgb.train(param, dtrain, num_round, watchlist)

[0]	eval-ndcg@5:1	train-ndcg@5:0.116932
[1]	eval-ndcg@5:1	train-ndcg@5:0.11943
[2]	eval-ndcg@5:1	train-ndcg@5:0.120965
[3]	eval-ndcg@5:1	train-ndcg@5:0.12162
[4]	eval-ndcg@5:1	train-ndcg@5:0.122976
[5]	eval-ndcg@5:1	train-ndcg@5:0.124404
[6]	eval-ndcg@5:1	train-ndcg@5:0.125901
[7]	eval-ndcg@5:1	train-ndcg@5:0.126952
[8]	eval-ndcg@5:1	train-ndcg@5:0.128664
[9]	eval-ndcg@5:1	train-ndcg@5:0.129868
[10]	eval-ndcg@5:1	train-ndcg@5:0.130471
[11]	eval-ndcg@5:1	train-ndcg@5:0.131269
[12]	eval-ndcg@5:1	train-ndcg@5:0.131692
[13]	eval-ndcg@5:1	train-ndcg@5:0.132212
[14]	eval-ndcg@5:1	train-ndcg@5:0.133477
[15]	eval-ndcg@5:1	train-ndcg@5:0.133932
[16]	eval-ndcg@5:1	train-ndcg@5:0.13434
[17]	eval-ndcg@5:1	train-ndcg@5:0.13549
[18]	eval-ndcg@5:1	train-ndcg@5:0.136416
[19]	eval-ndcg@5:1	train-ndcg@5:0.13632
[20]	eval-ndcg@5:1	train-ndcg@5:0.137206
[21]	eval-ndcg@5:1	train-ndcg@5:0.137652
[22]	eval-ndcg@5:1	train-ndcg@5:0.138182
[23]	eval-ndcg@5:1	train-ndcg@5:0.138433
[24]	eval-ndcg@5:1	train-ndcg@5

[199]	eval-ndcg@5:1	train-ndcg@5:0.155218
[200]	eval-ndcg@5:1	train-ndcg@5:0.155223
[201]	eval-ndcg@5:1	train-ndcg@5:0.155223
[202]	eval-ndcg@5:1	train-ndcg@5:0.155242
[203]	eval-ndcg@5:1	train-ndcg@5:0.155242
[204]	eval-ndcg@5:1	train-ndcg@5:0.155242
[205]	eval-ndcg@5:1	train-ndcg@5:0.155293
[206]	eval-ndcg@5:1	train-ndcg@5:0.155215
[207]	eval-ndcg@5:1	train-ndcg@5:0.155215
[208]	eval-ndcg@5:1	train-ndcg@5:0.155215
[209]	eval-ndcg@5:1	train-ndcg@5:0.155114
[210]	eval-ndcg@5:1	train-ndcg@5:0.155114
[211]	eval-ndcg@5:1	train-ndcg@5:0.155215
[212]	eval-ndcg@5:1	train-ndcg@5:0.155306
[213]	eval-ndcg@5:1	train-ndcg@5:0.155306
[214]	eval-ndcg@5:1	train-ndcg@5:0.155306
[215]	eval-ndcg@5:1	train-ndcg@5:0.155306
[216]	eval-ndcg@5:1	train-ndcg@5:0.155306
[217]	eval-ndcg@5:1	train-ndcg@5:0.155306
[218]	eval-ndcg@5:1	train-ndcg@5:0.155306
[219]	eval-ndcg@5:1	train-ndcg@5:0.155306
[220]	eval-ndcg@5:1	train-ndcg@5:0.155306
[221]	eval-ndcg@5:1	train-ndcg@5:0.155204
[222]	eval-ndcg@5:1	train-ndcg@5:0

[395]	eval-ndcg@5:1	train-ndcg@5:0.155416
[396]	eval-ndcg@5:1	train-ndcg@5:0.155416
[397]	eval-ndcg@5:1	train-ndcg@5:0.155416
[398]	eval-ndcg@5:1	train-ndcg@5:0.155416
[399]	eval-ndcg@5:1	train-ndcg@5:0.155416
[400]	eval-ndcg@5:1	train-ndcg@5:0.155416
[401]	eval-ndcg@5:1	train-ndcg@5:0.155416
[402]	eval-ndcg@5:1	train-ndcg@5:0.155416
[403]	eval-ndcg@5:1	train-ndcg@5:0.155416
[404]	eval-ndcg@5:1	train-ndcg@5:0.155416
[405]	eval-ndcg@5:1	train-ndcg@5:0.155416
[406]	eval-ndcg@5:1	train-ndcg@5:0.155416
[407]	eval-ndcg@5:1	train-ndcg@5:0.155416
[408]	eval-ndcg@5:1	train-ndcg@5:0.155416
[409]	eval-ndcg@5:1	train-ndcg@5:0.155416
[410]	eval-ndcg@5:1	train-ndcg@5:0.155416
[411]	eval-ndcg@5:1	train-ndcg@5:0.155416
[412]	eval-ndcg@5:1	train-ndcg@5:0.155416
[413]	eval-ndcg@5:1	train-ndcg@5:0.155416
[414]	eval-ndcg@5:1	train-ndcg@5:0.155416
[415]	eval-ndcg@5:1	train-ndcg@5:0.155416
[416]	eval-ndcg@5:1	train-ndcg@5:0.155416
[417]	eval-ndcg@5:1	train-ndcg@5:0.155416
[418]	eval-ndcg@5:1	train-ndcg@5:0

In [36]:
predictions = model.predict(dtest)

In [40]:
predictions[0:50]

array([-4.61346388, -2.62902498, -6.0048995 , -7.14047623, -7.51059341,
       -7.24228573, -1.00295293, -7.15137625, -5.95473385, -4.44423485,
        0.26970857, -7.56391525, -2.04789257, -2.51592636, -1.63177133,
       -3.63443327,  1.17775869, -3.34799314, -6.58916569, -7.29824018,
       -6.47084379, -6.31928396, -3.60964632,  1.29346585, -2.3839283 ,
       -1.53302574, -2.86201382, -1.73214769,  3.84289837, -4.78115702,
        0.35583115, -0.7107234 , -3.27604222,  3.14303112, -3.15917635,
       -7.96436882, -4.57471895, -0.18780178, -7.58180141,  1.3587296 ,
       -3.76664829, -4.9190526 , -4.82646799, -4.9190526 , -2.66047931,
       -6.58549595, -3.75814486, -1.96842837, -1.41662848, -7.81976986], dtype=float32)

In [43]:
i = 0
query_predictions = {}
for query_id, docs in test_queries:
    query_predictions[query_id] = np.array(predictions[i:i+len(docs)])
    i+=len(docs)

In [44]:
result = []
for query_id, docs in test_queries:
    top_docs = np.array(docs)[np.flip(np.argsort(query_predictions[query_id]),0)[0:5]]
    for doc_id in top_docs:
        result.append((query_id, doc_id))

In [51]:
df = pd.DataFrame(result, columns=['QueryId','DocID'])

In [53]:
df.to_csv('submit.csv', index=False)